# BASES DE DATOS CON PYTHON

![Python](https://memgraph.com/images/blog/in-memory-databases-that-work-great-with-python/cover.png)

### 1 - Introducción

Desde Python podemos interactuar con distintos tipos de bases de datos, ya sean **relacionales** o **no relacionales**. 

Para ello, existen distintas **librerías** que nos permiten realizar operaciones `CRUD` (lectura, escritura, actualización y borrado) sobre las bases de datos.

Vamos a ver las más comunes y más sencillas por las que podemos empezar a trabajar con bases de datos desde Python.

### 2 - MySQL Connector
Primera librería que vamos a ver es `mysql-connector-python`. Para instalarla, ejecutamos el siguiente comando:

```bash
!pip install mysql-connector-python
```

In [1]:
#instalamos la librería en caso de que no la tengamos
#!pip install mysql-connector-python

In [2]:
import mysql.connector as conn #Alias no estándar, pero muy usado (connection --> conexión)

Vamos a trabajar con `MySQL` en nuestro **localhost**, es decir una base de datos local en nuestro **propio ordenador**.

Para empezar a trabajar con MySQL desde Python tendremos que darle la información de :
- `Host`: Donde queremos conectarnos (ya sea local o remota)
- `Usuario` que se va a conectar
- `Credenciales` del usuario
- `Nombre` de la base de datos a la que nos queremos conectar (si nuestro usuario tiene acceso a varias bases de datos)
- `Puerto` de conexión (No siempre es necesario)

```python
mydb = conn.connect(
  host="localhost", #Si no es local, ponemos la IP de la máquina
  user="root",
  passwd ="contraseña", #Nuestra contraseña para acceder a mysql
  database="BasedeDatos" #Si queremos conectarnos a una base de datos en concreto, sino entraremos a la raíz
)
cursor=mydb.cursor() #Creamos un cursor para poder ejecutar comandos, un cursor es un objeto que nos permite interactuar con la base de datos
```

#### 2.1 - Como SÍ cargar los datos de la conexión

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [4]:
#Creamos fichero .env
#Comprobamos que lo leemos correctamente

Cuidado con cómo creamos el archivo .env:
```
variable=valor_de_la_variable
```

Sin espacios, sin comas (a no ser que las comas formen parte del valor de la variable) y con cuidado con los comentarios.

In [5]:
#Ahora puedo montar mi motor sin exponer credenciales ni detalles de la conexión, lo cargamos todo desde el fichero dotenv
mydb = conn.connect(
    host = os.getenv('host'),
    password = os.getenv('pwd'),
    #database = os.getenv('db'),
    user = os.getenv('usuario'),
    port = os.getenv('puerto')
)
cursor = mydb.cursor()

In [6]:
#Vamos a comprobar si estamos bien conectados:
cursor.execute("SHOW DATABASES")

In [ ]:
for x in cursor:
    print(x)

#### 2.2 - CREANDO UNA BASE DE DATOS

In [8]:
#Vamos a crear una base de datos y la llamaremos "prueba"
cursor.execute("CREATE DATABASE prueba") # siempre es mejor utilizar la sentencia CREATE DATABASE IF NOT EXISTS nombre_base_datos

Vamos a comprobar si todo ha funcionado correctamente, vamos a **conectarnos** a la base de datos que acabamos de crear que se llama **"prueba"**.

In [9]:
cursor.execute("USE prueba") #Nos cambiamos a la base de datos prueba
cursor.execute("SHOW TABLES")
for x in cursor:
    print(x) #No hay tablas

Efectivamente la base de datos `prueba` existe, pero está vacía, no tiene ninguna tabla. Por lo que vamos a crearla.

Vamos a crear una tabla de alumnos

In [ ]:
import pandas as pd

# Alumnos de la clase:
alumnos = [{'alumno':'Gino','sexo':'hombre','ciudad':'madrid'},
           {'alumno':'Yovanna','sexo':'mujer','ciudad':'madrid'},
           {'alumno':'Daniela','sexo':'mujer','ciudad':'madrid'},
           {'alumno':'Zuley','sexo':'hombre','ciudad':'madrid'},
           {'alumno':'Yan','sexo':'hombre','ciudad':'madrid'},
           {'alumno':'Rubén','sexo':'hombre','ciudad':'madrid'},
           {'alumno':'JJ','sexo':'hombre','ciudad':'madrid'},
           {'alumno':'Tobal','sexo':'hombre','ciudad':'madrid'}]

df_alumnos = pd.DataFrame(alumnos)
df_alumnos

Ahora vamos a crear una tabla de ejercicios

In [ ]:
#ejercicios va a ser una lista de notas aleatorias de los alumnos
import pandas as pd
def notas():
    import random
    return [random.randint(1,10) for i in range(5)]
ejercicios = [{'alumno': 'Gino', 'ejercicios': notas()},
              {'alumno': 'Yovanna','ejercicios': notas()},
              {'alumno': 'Daniela','ejercicios': notas()},
              {'alumno': 'Zuley','ejercicios': notas()},
              {'alumno': 'Yan','ejercicios': notas()},
              {'alumno': 'Rubén','ejercicios': notas()},
              {'alumno': 'JJ','ejercicios': notas()},
              {'alumno': 'Tobal','ejercicios': notas()}]
df_ejercicios = pd.DataFrame(ejercicios)
df_ejercicios

In [ ]:
df_ejercicios = df_ejercicios.explode('ejercicios')
df_ejercicios.head()

In [ ]:
df_ejercicios.reset_index(drop=True,inplace=True)
df_ejercicios.head(10)

Ahora que tenemos los 2 dataframes con los datos que queremos, vamos a crear las tablas en nuestra base de datos.

1 - Tabla `alumnos`:

In [14]:
#Vamos a crear una base de datos llamada "alumnos_data" con los datos de df_alumnos
cursor.execute("CREATE DATABASE IF NOT EXISTS alumnos_data")
cursor.execute("USE alumnos_data")

#Entidad alumnos --> alumno | sexo | ciudad

In [15]:
#Entidad ejercicios --> alumno | ejercicios

Ahora que tenemos nuestras 2 tablas creadas en nuestra base de datos `alumnos_data`, vamos a insertar los datos que tenemos en nuestros dataframes. El de alumnos y el de ejercicios.

In [ ]:
df_alumnos.head(5) #Refresecamos que pinta tiene el df

Refrescamos como era una onstrucción INSERT de SQL:

```sql
--Opcion 1
INSERT INTO nombre_tabla (columna1, columna2, columna3, ...)
VALUES (valor1, valor2, valor3, ...);
```

```sql
--Opcion 2
INSERT INTO nombre_tabla (columna1, columna2, columna3, ...)
VALUES (valor1, valor2, valor3, ...),
       (valor1, valor2, valor3, ...),
       (valor1, valor2, valor3, ...),
       ...
       (valor1, valor2, valor3, ...);
```

```sql
--Opcion 3
INSERT INTO nombre_tabla
VALUES (valor1, valor2, valor3, ...);
```

In [17]:
#Esto lo codeamos en clase

In [18]:
#Vamos a hacer lo mismo pero en una sola query

In [19]:
#Truncamos la tabla 


In [ ]:
df_ejercicios.head() #Refresecamos que pinta tiene el df

In [21]:
#Poblamos la tabla de ejercicios

Pues de esta manera podemos cargar nuestros datos y hacernos nuestras propias databases y trabajar con ellas desde Python, haciendo lo mismo que haríamos en MySQL pero sin tener que salirnos de Python y andar cambiando de Workbench (o el que sea) a Python.

También podríamos hacer el proceso inverso y sacar datos de SQL y meterlo en un dataframe de Pandas

#### 2.3 - CONSULTANDO LA BASE DE DATOS

In [ ]:
import time
cursor.execute("SELECT * FROM alumnos")
time.sleep(1)
for x in cursor:
    time.sleep(1)
    print(x)#,end='\r') #para que sobreescriba encima

In [ ]:
cursor.close()

Ahora es vuestro turno, vamos a pegarnos con este [csv](https://drive.google.com/file/d/1tOg8MGCj8NZpxshozKUu7QpDuS4owsGS/view?usp=drive_link)

In [25]:
mydb = conn.connect(
    host = os.getenv('host'),
    password = os.getenv('pwd'),
    database = os.getenv('db'),
    user = os.getenv('usuario'),
)
cursor = mydb.cursor()

In [ ]:
import pandas as pd
df = pd.read_csv('apple_store.csv')
df.head()

In [ ]:
df.shape

In [28]:
#Vamos a crear una tabla en nuestra base de datos
cursor.execute("CREATE DATABASE IF NOT EXISTS apple_store")
#Vamos a usarla
cursor.execute("USE apple_store")

In [30]:
query = """
CREATE TABLE IF NOT EXISTS apple (
  id BIGINT,
  track_name VARCHAR(300),
  size_bytes BIGINT,
  price FLOAT(5,2),
  rating_count_tot BIGINT,
  rating_count_ver BIGINT,
  user_rating FLOAT(5,2),
  user_rating_ver FLOAT(5,2),
  prime_genre VARCHAR(100),
  PRIMARY KEY (id)
  )
  """

In [31]:
cursor.execute(query)

In [ ]:
cursor.execute("SHOW TABLES")
for x in cursor:
    print(x)
#Vamos a insertas los datos de df en la tabla apple

Al intentar insertar las filas hay problemas con determinados caracteres, como por ejemplo `'` pues lo que hace es indicar que ahí termina una cadena de texto.

In [33]:
#Vaciamos la tabla 
cursor.execute("TRUNCATE TABLE apple")

Para evitar este conflicto hay otra manera de realizar la inserción:

In [ ]:
for i, row in df.iterrows():
    query = """
    INSERT INTO apple VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # recordad que si se introducen la misma cantidad de valores que columnas no hace falta especificar las columnas
    values = (
        row['id'],
        row['track_name'],
        row['size_bytes'],
        row['price'],
        row['rating_count_tot'],
        row['rating_count_ver'],
        row['user_rating'],
        row['user_rating_ver'],
        row['prime_genre']
    )
    print(query, values)
    cursor.execute(query, values)
    cursor.execute("COMMIT") #Sino los cambios no se guardan, no se ejecutan realmente
    #cursor.close()
(print(f"{i+1} filas insertadas"))

En este caso, utilizas `%s` como marcadores de posición para los valores en la consulta `SQL` y luego pasas una tupla `values` que contiene los valores reales a `cursor.execute()`. De esta manera, el driver de MySQL se encargará de escapar correctamente todos los valores para prevenir errores de sintaxis y vulnerabilidades de seguridad.

In [ ]:
cursor.close()

# 3 - SQL ALCHEMY

Sql Alchemy es una librería que nos permite trabajar con bases de datos de una manera más sencilla y más parecida a como lo hacemos en SQL.

Es la librería más usada para trabajar con bases de datos desde Python, ya que nos permite trabajar con cualquier base de datos, tanto SQL como NoSQL.Aunque para trabajar con bases de datos NoSQL existen otras librerías más específicas, mejores y más sencillas en esos casos.

## 3.1 - CONFIGURACIÓN CONEXIÓN:

Para empezar a trabajar con SQL Alchemy, lo primero que tenemos que hacer es instalarla:
```bash
!pip install sqlalchemy
```
Lo siguiente es configurar la `conexión` a nuestra base de datos, creando lo que se llama un `engine`(motor) y pasándole la información de la conexión en una `string de conexión`.

In [36]:
#%pip install sqlalchemy #Instalamos la librería
import sqlalchemy as sa

In [37]:
#Parametros de la conexion:
host = os.getenv('host')
password = os.getenv('pwd')
database = os.getenv('db')
user = os.getenv('usuario')
engine = sa.create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}') #Creamos el engine

In [ ]:
#testeamos la conexión
engine.connect()

Podemos utilizar nuestro conector sin especificar una base de datos, en ese caso cómo creamos nuestro engine debe tener una pequeña modificación.
```python
#Parametros de la conexion:
host = os.getenv('host')
password = os.getenv('password')
user = os.getenv('user')
engine = sa.create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/') #Creamos el engine sin especificar una base de datos
```

In [39]:
#Parametros de la conexion:
host = os.getenv('host')
password = os.getenv('pwd')
user = os.getenv('usuario')
engine = sa.create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/') #Creamos el engine

In [ ]:
#Probamos que conecta correctamente
engine.connect() #Mi motor puede crear una conexion

Ya tenemos creada nuestra base de datos con una única tabla con aplicaciones de apple_store. Para poder ver su contenido lo primero que vamos a tener que hacer es conectarnos a esa base de datos:
```sql
--Activar una base de datos para poder hacer consultas en ella:
USE nombre_base_datos;
```

In [ ]:
#Vamos a utilizar la base de datos
query = 'USE apple_store'
#Vamos a recuperar los datos de la tabla apple
query_test = 'SELECT * FROM apple LIMIT 10'

with engine.connect() as con:
    con.execute(sa.text(query))
    test = con.execute(sa.text(query_test))
    print(test.fetchall())

In [42]:
for fila in test.fetchall():
    print(fila)

In [ ]:
#Vamos a ver que tablas hay en la base de datos
query = "SHOW TABLES"

with engine.connect() as con: #Con esto nos conectamos a la base de datos
    con.execute(sa.text(query)) #Con esto ejecutamos la query
    print(con.execute(sa.text(query)).fetchall()) #Con esto nos devuelve el resultado de la query

In [ ]:
# Solo hay una tabla, así que vamos a ver todo lo que tiene esa tabla
query = "SELECT * FROM apple"

with engine.connect() as con:
    con.execute(sa.text(query)) #Con esto ejecutamos la query
    print(con.execute(sa.text(query)).fetchall()) #Con esto nos devuelve el resultado de la query

Es un poco complicada la **lectura** así que lo que podemos hacer de una manera muy rápida es convertir esta **salida** de SQL en un **DataFrame**. Hay varios métodos para hacerlo:
1. Se puede utilizar la función de **pandas** `read_sql_query()`
2. Se puede utilizar la query directamente como data para la función de crear un DataFrame `pd.DataFrame`

In [ ]:
# Primer caso:
query = "SELECT * FROM apple"

with engine.connect() as con:
    df_apple = pd.read_sql_query(query # query que quiero ejecutar
                                 ,con) # conexión a la base de datos

print(f'El tamaño de esta salida es: {df_apple.shape}')
df_apple

In [ ]:
# Segundo caso:
query = "SELECT * FROM apple"

with engine.connect() as con:
    df_apple = pd.DataFrame(con.execute(sa.text(query)).fetchall()) # le digo que data es directamente la query

print(f'El tamaño de esta salida es: {df_apple.shape}')
df_apple

Hemos visto cómo recorrer un DataFrame y crear una tabla en SQL dentro de una base de datos de una manera dinámica con `f-strings`. 

No obstante, hay **otras maneras** de hacerlo, con SQLAlchemy podemos directamente convertir un **DataFrame** en una **tabla**, sin necesidad de preocuparnos por hacer algún tipo de **transformación previa**.

In [47]:
# Primero debemos crear una tabla para poder introducir la información
# Una vez creada la tabla con las columnas necesarias ya podremos introducir la información
query = """
CREATE TABLE IF NOT EXISTS apple_short (
    id BIGINT,
    price FLOAT(5,2),
    user_rating FLOAT(5,2),
    PRIMARY KEY (id)
    )
    """

with engine.connect() as con:
    con.execute(sa.text(query))

In [49]:
# Primero, para no tener dos tablas idénticas, voy a filtrar el DataFrame para tener únicamente id, precio y puntuación.
df_apple_insert = df_apple[['id', 'price', 'user_rating']]

# Usamos el conector para introducir este df como una nueva tabla
with engine.connect() as con:
    df_apple_insert.to_sql('apple_recortada', # nombre de la tabla 
                            con=con, # conexión a la base de datos 
                            schema='apple_store', # nombre de la base de datos
                            if_exists='replace', # comportamiento si la tabla ya existe ['fail', 'replace', 'append']
                            index=False # si quiero añadir el índice como columna
                            )

In [ ]:
# Comprobamos los 5 primeros registros para ver que se ha insertado información
query = """
SELECT *
FROM apple_recortada
LIMIT 5
"""

with engine.connect() as con:
    con.execute(sa.text(query))
    for result in con.execute(sa.text(query)):
        print(result)

## 3.2 - Creamos una tabla en nuestra base de datos a partir de un DataFrame

Esta vez vamos a hacerlo, pero sin necesidad de crear la tabla en la base de datos, vamos a hacerlo directamente desde el DataFrame.

In [ ]:
df_apple.head()

In [ ]:
df_apple_directa = df_apple[['track_name', 'size_bytes']]
df_apple_directa

In [53]:
# Usamos el conector para introducir este df como una nueva tabla
with engine.connect() as con:
    df_apple_directa.to_sql('apple_directa', # nombre de la tabla 
                            con=con, # conexión a la base de datos 
                            schema='apple_store', # nombre de la base de datos
                            if_exists='replace', # comportamiento si la tabla ya existe ['fail', 'replace', 'append']
                            index=False # si quiero añadir el índice como columna
                            )

In [ ]:
#Comprobamos
query = 'SELECT * FROM apple_directa LIMIT 5'

with engine.connect() as con:
    con.execute(sa.text(query))
    for result in con.execute(sa.text(query)):
        print(result)

## 3.3 - Actualizando registros en la base de datos

Vamos a ver cómo podemos **actualizar** registros en nuestra base de datos. Para ello, vamos a hacer una **actualización** de la tabla `apple_store` que hemos creado anteriormente.

In [58]:
#Vamos descargar el contenido de la tabla apple en un DataFrame
#para cambiar los datos de la columna 'prime_genre' a mayúsculas y 
# actualizar la tabla en la base de datos
query = "SELECT * FROM apple"

with engine.connect() as con:
    df_apple = pd.read_sql_query(query, con)
    

In [ ]:
df_apple[['prime_genre']]

In [ ]:
df_apple['prime_genre'] = df_apple['prime_genre'].str.lower()
df_apple[['prime_genre']]

In [62]:
# Cargamos el DataFrame en la base de datos
with engine.connect() as con:
    df_apple.to_sql('apple', # nombre de la tabla 
                    con=con, # conexión a la base de datos 
                    schema='apple_store', # nombre de la base de datos
                    if_exists='replace', # comportamiento si la tabla ya existe ['fail', 'replace', 'append']
                    index=False # si quiero añadir el índice como columna
                    )

In [ ]:
query = "SELECT * FROM apple LIMIT 10"

with engine.connect() as con:
    con.execute(sa.text(query))
    df = pd.read_sql_query(query, con)
df[['prime_genre']]

In [ ]:
#Ejercicio 
# Vamos a resetear la columna id, para que empiece en 1 hasta el final
#size_bytes vamos a cambiarlo a MB (1 MB son 1024*1024 bytes) y redondeamos a 2 decimales
# vamos a crear una nueva columna que sea price . rating_count_tot y la llamaremos income